In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
import re
from matplotlib.pyplot import figure, show, axes, sci
from matplotlib import cm, colors
from matplotlib.font_manager import FontProperties
from numpy import amin, amax, ravel
import numpy.ma as ma

In [2]:
matrix = np.zeros( (3,3,9) )
data = open('/Users/yao/Google Drive/data/216/ratio_paper_81','r')

In [3]:
value = []
lines = data.readlines()
for line in lines:
    match = re.match(r"([a-z]+)([ ]+)([a-z]+)([ ]+)([a-z]+)([ ]+)([0-9.]+)([ ]+)([0-9.]+)", line, re.I)
    if match:
        items = match.groups()
        value.append(float(items[6]))

In [36]:
#for i in range(len(value_eg)):
#    if value_eg[i] >= 1:
#        value_eg[i] = 1

In [4]:
index = 0
for a in range(3):
    for b in range(9):
        for c in range(3):
            matrix[c][a][b] = value[index]
            index = index + 1

In [5]:
masked_matrix = ma.masked_where(matrix>2, matrix)

In [6]:
fig = figure()
#cmap = cm.cool
w = 1.2
h = 0.15
ax = []
images = []
vmin = 1e40
vmax = -1e40
cax = fig.add_axes([0.92, 0.2, 0.02, 0.5])

In [7]:
for i in range(3):
    pos = [0, 0.56 - i*1.2*h, w, h]
    a = fig.add_axes(pos)
    plt.yticks(np.arange(3),['K','Rb','Cs'], size='small')
    plt.xticks(np.arange(9),[])
    if i == 2: 
        plt.xticks(np.arange(9),['Si','Ge','Sn','Pb','Ni','Pd','Pt','Se','Te'],size='small')
    dd = ravel(masked_matrix[i])
    # Manually find the min and max of all colors for
    # use in setting the color scale.
    vmin = min(vmin, amin(dd))
    vmax = max(vmax, amax(dd))
    images.append(a.imshow(masked_matrix[i],interpolation = 'nearest',aspect=1))

    ax.append(a)

In [8]:
# Set the first image as the master, with all the others
# observing it for changes in cmap or norm.

class ImageFollower:
    'update image in response to changes in clim or cmap on another image'
    def __init__(self, follower):
        self.follower = follower
    def __call__(self, leader):
        self.follower.set_cmap(leader.get_cmap())
        self.follower.set_clim(leader.get_clim())
norm = colors.Normalize(vmin=vmin, vmax=vmax)
for i, im in enumerate(images):
    im.set_norm(norm)
    if i > 0:
        images[0].callbacksSM.connect('changed', ImageFollower(im))
fig.colorbar(images[0], cax)
axes(ax[0])     # Return the current axes to the first one,
sci(images[0])  # because the current image must be in current axes.
show()

/usr/local/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
